In [ ]:
!pip install pdfplumber
!pip install spacy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 55.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:

import os
import pdfplumber
import spacy
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import re

# Mount Google Drive (run this only in Google Colab)
from google.colab import drive
drive.mount('/content/drive')

# Specify the folder path where the resumes are stored in Google Drive
folder_path = '/content/drive/My Drive/Resumes/'  # Replace with your actual folder path

# Load spaCy model for Named Entity Recognition (NER)
nlp = spacy.load("en_core_web_sm")

# Define a list of skills (expand this list as needed)
skills_list = [
    "Java", "Spring", "Hibernate", "RESTful APIs", "SQL", "Manual Testing", "Selenium", "JIRA",
"Bug Tracking", "AWS", "Docker", "Kubernetes", "Jenkins", "Terraform", "Python", "Django",
"Flask", "Pandas", "HTML", "CSS", "JavaScript", "Adobe Photoshop", "UI/UX", "Recruitment",
"Employee Relations", "Payroll", "Performance Management", "Hadoop", "Hive", "MapReduce",
"Spark", "Scala", "Blockchain", "Solidity", "Ethereum", "Smart Contracts", "ETL Tools (Informatica)",
"Data Warehousing", "SSIS", "Process Improvement", "Lean Six Sigma", "Supply Chain", "Project Management",
"R", "Machine Learning", "Data Mining", "TensorFlow", "Sales Strategy", "CRM", "Lead Generation",
"Market Analysis", "Salesforce", "CAD", "SolidWorks", "Keras", "Thermodynamics", "Manufacturing Processes",
"Oracle", "MySQL", "Database Design", "Performance Tuning", "Circuit Design", "MATLAB", "Power Systems",
"PLC Programming", "Requirements Gathering", "Business Process Modeling", "Agile Methodologies", "C",
"C++", "ASP.NET", "MVC", "SQL Server", "Entity Framework", "Test Automation", "Load Testing",
"Network Security", "Firewalls", "IDS/IPS", "VPN", "Wireshark", "CEH", "SAP ABAP", "SAP HANA",
"FICO", "BAPI", "AutoCAD", "Revit", "Structural Analysis", "Construction Management", "Technical Support",
"Windows/Linux", "Troubleshooting", "Networking", "Cloud Computing", "Azure", "Deep Learning", "NLP",
"UI Design", "UX Design", "Figma", "Adobe XD", "Prototyping", "Sketch", "Cybersecurity", "Risk Management",
"Threat Hunting", "SIEM", "Incident Response", "Android/iOS Development", "Kotlin", "Swift", "Flutter",
"MS Project", "Stakeholder Communication", "React", "Vue.js", "IT Strategy", "Systems Analysis",
"Client Communication", "Excel", "Data Visualization (Tableau, Power BI)", "Computer Vision",
"3D Modeling", "Power BI", "Tableau", "Game Physics", "CI/CD", "Security Automation", "Embedded Systems", "IoT Protocols",
"MQTT", "Raspberry Pi", "Cisco Networking", "Routing", "Switching", "Firewall Configuration",
"Vulnerability Assessment", "Cyber Forensics", "Robot Operating System (ROS)", "Mechatronics",
"Penetration Testing", "Ethical Hacking", "Kali Linux", "Metasploit", "IT Auditing",
"Compliance (SOX, GDPR)", "Big Data", "Unity", "VR Design", "Cloud Security", "DevSecOps",
"Technical Writing", "Documentation", "API Documentation", "Editing", "XML", "Security Assessments",
"Compliance (ISO 27001, PCI-DSS)", "jQuery", "Stored Procedures", "Microservices",
"Technical Troubleshooting", "Customer Service", "Product Lifecycle Management",
"Embedded C", "Image Processing", "IT Training", "Instructional Design", "e-Learning",
"Survey Design", "Software Support", "Game Design", "Storyboarding", "Level Design",
"SCADA", "GCP", "Blockchain", "Hyperledger", "IoT Solutions", "Cloud Integration",
"Adobe Creative Suite", "Cyber Forensics", "Sales", "Marketing", "Data Engineering",
"Data Science", "Machine Learning", "Deep Learning", "Natural Language Processing",
"Computer Vision", "Data Visualization", "Data Analysis", "Data Warehousing", "ETL",
"Big Data", "Cloud Computing", "DevOps", "Software Development", "Web Development",
"Mobile Development", "Game Development", "Security", "Networking", "Systems Administration"
]



# Function to extract skills from resume text
def extract_skills(text):
    found_skills = [skill for skill in skills_list if skill.lower() in text.lower()]
    return ', '.join(found_skills)  # Join as string for storing in DataFrame

# Function to extract candidate identity (name, email, phone)
def extract_identity(resume_text):
    lines = resume_text.split('\n')
    name = lines[0].strip()  # First line as candidate's name
    email = None
    phone = None
    for line in lines:
        if "@" in line:  # Basic way to identify the email
            email = line.strip()
        if re.search(r'\b\d{10}\b', line):  # Basic regex for 10-digit phone number
            phone = line.strip()
    return name, email, phone

# Function to extract years of experience from text
def extract_experience(text):
    # Regex to find patterns like "3 years", "5+ years", etc.
    match = re.search(r'(\d+)\s*(?:\+?\s*years?|yrs)', text.lower())
    if match:
        return int(match.group(1))
    return 0  # Default to 0 if no experience is found

# List to store resume details
resume_data = []

# Loop through all PDF files in the Google Drive folder
for filename in os.listdir(folder_path):
    if filename.endswith('.pdf'):
        file_path = os.path.join(folder_path, filename)
        with pdfplumber.open(file_path) as pdf:
            text = ''
            for page in pdf.pages:
                text += page.extract_text() or ''

        # Extract information from the resume
        name, email, phone = extract_identity(text)
        skills = extract_skills(text)
        experience = extract_experience(text)

        # Store the extracted information in a dictionary
        resume_data.append({
            'filename': filename,
            'name': name,
            'email': email,
            'phone': phone,
            'skills': skills,
            'experience': experience,
            'selected': False  # Initially mark all as not selected
        })

# Convert resume data to a DataFrame
resume_df = pd.DataFrame(resume_data)

# Save the resume data to a CSV file for future tracking
output_path = '/content/drive/My Drive/processed_resumes.csv'
resume_df.to_csv(output_path, index=False)

# Ensure Experience column is numeric
resume_df['experience'] = resume_df['experience'].fillna(0).astype(int)

# Vectorize Skills using TF-IDF
vectorizer = TfidfVectorizer()
skills_vectors = vectorizer.fit_transform(resume_df['skills'].fillna(''))

# Combine skills vectors with experience
skills_df = pd.DataFrame(skills_vectors.toarray())
skills_df['experience'] = resume_df['experience']

# Ensure all columns are numeric and have string names
X = skills_df.astype(float)
X.columns = X.columns.astype(str)

# Standardize the data before DBSCAN
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply DBSCAN clustering
dbscan = DBSCAN(eps=0.5, min_samples=2)  # You can adjust eps and min_samples
resume_df['Cluster'] = dbscan.fit_predict(X_scaled)

# Function to process experience input
def process_experience(exp_input):
    exp_match = re.search(r'(at least|at most|exact|)\s*(\d+)', exp_input.lower())
    if exp_match:
        condition = exp_match.group(1) or 'exact'
        years = int(exp_match.group(2))
    else:
        raise ValueError("Invalid experience input format. Please specify 'at least', 'at most', or 'exact'.")
    return condition, years

# Function to extract email and phone from resume text using regex
def extract_identity(resume_text):
    # Use regex for email extraction
    email_match = re.search(r'[\w\.-]+@[\w\.-]+', resume_text)
    email = email_match.group(0) if email_match else 'Not Found'

    # Use regex for phone number extraction (international or standard formats)
    phone_match = re.search(r'\+?\d[\d\-\.\s]+\d', resume_text)
    phone = phone_match.group(0) if phone_match else 'Not Found'

    return email, phone

# Filter candidates who meet both skills and experience criteria
def filter_candidates(cluster_candidates, skills_required, experience_condition, experience_years):
    def has_all_skills(candidate_skills, required_skills):
        candidate_skills_set = set(map(str.lower, candidate_skills.split(', ')))
        return all(skill.lower() in candidate_skills_set for skill in required_skills)

    # Filter based on skills: keep only candidates with all required skills
    filtered_candidates = cluster_candidates[
        cluster_candidates['skills'].apply(lambda x: has_all_skills(x, skills_required))
    ]

    # Apply strict experience condition
    if experience_condition == 'at least':
        filtered_candidates = filtered_candidates[filtered_candidates['experience'] >= experience_years]
    elif experience_condition == 'at most':
        filtered_candidates = filtered_candidates[filtered_candidates['experience'] <= experience_years]
    elif experience_condition == 'exact':
        filtered_candidates = filtered_candidates[filtered_candidates['experience'] == experience_years]

    return filtered_candidates[['name', 'email', 'phone', 'skills', 'experience']]

# Match candidates using DBSCAN
def match_candidates_with_dbscan(skills_required, exp_input, resume_df, vectorizer, scaler, dbscan, X_scaled):
    # Process experience input
    experience_condition, experience_years = process_experience(exp_input)

    # Vectorize query skills
    query_vector = vectorizer.transform([', '.join(skills_required)]).toarray()

    # Add experience to the query vector
    query_vector = np.concatenate([query_vector, np.array([[experience_years]])], axis=1)

    # Scale the query vector
    query_vector_scaled = scaler.transform(query_vector)

    # Calculate cosine similarity between query vector and all vectors in X_scaled
    similarities = cosine_similarity(query_vector_scaled, X_scaled)

    # Get the cluster of the most similar candidate
    most_similar_index = np.argmax(similarities)
    most_similar_cluster = resume_df.iloc[most_similar_index]['Cluster']

    # Filter candidates in the same cluster
    cluster_candidates = resume_df[resume_df['Cluster'] == most_similar_cluster]

    # Filter candidates who have all required skills and meet the experience condition
    return filter_candidates(cluster_candidates, skills_required, experience_condition, experience_years)

# Example inputs from the user
skills_input = input("Enter required skills (comma-separated): ")
experience_input = input("Enter experience requirement (e.g., 'at least 3 years', 'at most 3 years'): ")

# Convert skills input to list
skills_required = [skill.strip().lower() for skill in skills_input.split(',')]

# Find matching candidates
matching_candidates = match_candidates_with_dbscan(skills_required, experience_input, resume_df, vectorizer, scaler, dbscan, X_scaled)

# Display matching candidates or a message if none are found
if matching_candidates.empty:
    print("No matching candidates found based on the provided skills and experience.")
else:
    print("Matching Candidates:")
    print(matching_candidates)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Enter required skills (comma-separated): sql
Enter experience requirement (e.g., 'at least 3 years', 'at most 3 years'): 0
Matching Candidates:
                        name  \
0  Pratik Parasharam Ghevade   
4      Nawab Sikandar Shaikh   
7                 Vinay More   
8      SURAJ BALASAHEB KORDE   
9                YASH SAWANT   

                                               email  \
0                          pratikghevade02@gmail.com   
4                          nawabshaikh1617@gmail.com   
7  +91-8766700242 | mailto: morevinay87.1@gmail.c...   
8  surajkorde6608@gmail.com | 9075268375 | Rajgur...   
9  yashsawant25252525@ Dedicated M.Sc. Statistics...   

                                               phone  \
0                                               None   
4                                         8830282357   
7  +91-8766700242 | mailto: m

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
